# 1. Importing all necessary libs and modules

In [1]:
import pandas as pd
import numpy as np
import os 
import re
import matplotlib.pyplot as plt
import scipy
import os
from pathlib import Path
import math
from statsmodels.stats.multitest import multipletests
import statsmodels
from scipy.stats import ttest_rel, ttest_ind
from scipy import stats
from statsmodels.stats.multitest import fdrcorrection as fdr

# 2. Reading and filtering the data

In [2]:
# Reading the original normal data
normal =pd.read_csv('data/lusc-rsem-fpkm-tcga_paired.txt', sep="\t", header=0)
print(len(normal))
print(normal.columns)
print(normal.head(3))

19648
Index(['Hugo_Symbol', 'Entrez_Gene_Id', 'TCGA-43-7657', 'TCGA-58-8386',
       'TCGA-22-5478', 'TCGA-22-5472', 'TCGA-43-5670', 'TCGA-60-2709',
       'TCGA-22-5489', 'TCGA-77-8007', 'TCGA-22-5471', 'TCGA-22-4609',
       'TCGA-22-5482', 'TCGA-56-8082', 'TCGA-22-5483', 'TCGA-56-8623',
       'TCGA-33-4587', 'TCGA-56-7579', 'TCGA-43-3394', 'TCGA-34-8454',
       'TCGA-77-7338', 'TCGA-43-6143', 'TCGA-43-6773', 'TCGA-51-4080',
       'TCGA-34-7107', 'TCGA-39-5040', 'TCGA-43-6771', 'TCGA-92-7340',
       'TCGA-77-7138', 'TCGA-77-7142', 'TCGA-56-7823', 'TCGA-22-5491',
       'TCGA-22-5481', 'TCGA-90-7767', 'TCGA-77-8008', 'TCGA-56-7730',
       'TCGA-33-6737', 'TCGA-56-7582', 'TCGA-56-7731', 'TCGA-77-7337',
       'TCGA-43-7658', 'TCGA-56-8201', 'TCGA-85-7710', 'TCGA-56-7580',
       'TCGA-43-6647', 'TCGA-90-6837', 'TCGA-56-8083', 'TCGA-51-4079',
       'TCGA-56-7222', 'TCGA-22-4593', 'TCGA-51-4081', 'TCGA-56-8309'],
      dtype='object')
  Hugo_Symbol  Entrez_Gene_Id  TCGA-43-7657  TC

In [3]:
# Reading the original cancer data
cancer = pd.read_csv('data/lusc-rsem-fpkm-tcga-t_paired.txt', sep="\t", header=0)
print(len(cancer))
print(cancer.columns)
print(cancer.head(3))

19648
Index(['Hugo_Symbol', 'Entrez_Gene_Id', 'TCGA-43-7657', 'TCGA-58-8386',
       'TCGA-22-5478', 'TCGA-22-5472', 'TCGA-43-5670', 'TCGA-60-2709',
       'TCGA-22-5489', 'TCGA-77-8007', 'TCGA-22-5471', 'TCGA-22-4609',
       'TCGA-22-5482', 'TCGA-56-8082', 'TCGA-22-5483', 'TCGA-56-8623',
       'TCGA-33-4587', 'TCGA-56-7579', 'TCGA-43-3394', 'TCGA-34-8454',
       'TCGA-77-7338', 'TCGA-43-6143', 'TCGA-43-6773', 'TCGA-51-4080',
       'TCGA-34-7107', 'TCGA-39-5040', 'TCGA-43-6771', 'TCGA-92-7340',
       'TCGA-77-7138', 'TCGA-77-7142', 'TCGA-56-7823', 'TCGA-22-5491',
       'TCGA-22-5481', 'TCGA-90-7767', 'TCGA-77-8008', 'TCGA-56-7730',
       'TCGA-33-6737', 'TCGA-56-7582', 'TCGA-56-7731', 'TCGA-77-7337',
       'TCGA-43-7658', 'TCGA-56-8201', 'TCGA-85-7710', 'TCGA-56-7580',
       'TCGA-43-6647', 'TCGA-90-6837', 'TCGA-56-8083', 'TCGA-51-4079',
       'TCGA-56-7222', 'TCGA-22-4593', 'TCGA-51-4081', 'TCGA-56-8309'],
      dtype='object')
  Hugo_Symbol  Entrez_Gene_Id  TCGA-43-7657  TC

In [4]:
# A function to check if 2 dataframes have the sama genes(columns) and the same length
def check(normal, cancer):
    if not (normal.columns == cancer.columns).all():
        print('Different Columns')
    if not (len(normal) == len(cancer)):
        print('Different length')
    else: 
        print('checked')
check(normal, cancer)

checked


In [5]:
# Execluding rows where number of zeros is more than 25
normal_x = normal[((normal == 0).astype(int).sum(axis=1))<25]
cancer_x = cancer[((cancer == 0).astype(int).sum(axis=1))<25]
print(len(normal_x), len(cancer_x))

17626 17717


In [6]:
# Getting only the intersection between filtered normal and cancer data with zeros less than 25
names = np.intersect1d(normal_x.Hugo_Symbol, cancer_x.Hugo_Symbol)

# We fet the intersection by each genes name
normal_x = normal_x[normal_x['Hugo_Symbol'].isin(names)]
cancer_x = cancer_x[cancer_x['Hugo_Symbol'].isin(names)]

# Droping NAN values
normal_x.dropna()
cancer_x.dropna()

# Print the length and check if the 2 dataframes are identical after filteration
print(len(normal_x), len(cancer_x))
check(normal_x,cancer_x)

17275 17275
checked


In [7]:
# # Saving the filtered sheets
# normal_x.to_csv("filtered/normal.csv")
# cancer_x.to_csv("filtered/cancer.csv")

In [8]:
np.array(normal_x.iloc[0][2:])

array([62.12, 130.6, 33.06, 35.5, 73.03, 60.39, 92.05, 66.65, 54.33,
       15.56, 55.49, 30.34, 14.45, 100.83, 131.51, 59.55, 87.03, 71.5,
       23.08, 48.87, 72.52, 115.97, 45.21, 74.58, 8.92, 27.44, 40.64,
       35.76, 59.13, 20.86, 37.05, 48.18, 51.71, 49.56, 72.01, 21.94,
       27.44, 35.0, 76.17, 28.86, 90.77, 59.55, 40.07, 22.92, 29.91,
       82.29, 4.7, 37.32, 43.63, 77.25], dtype=object)

# 3. Normality test 

## 1. paired 

- Since the data is paired, we can check the normality on the difference between healthy and cancerous data for the same gene (The difference between every two corresponding rows in both dataframes)

In [9]:
# Create the difference dataframe(Execluding the names and IDs)
diff = normal_x.iloc[:,2:] - cancer_x.iloc[:,2:]

# Create a coulmn to hold the normality result for each sample
diff ['Normality'] = ''

# Check for tha shape
print(diff.shape)
diff.head(5)

(17275, 51)


,TCGA-43-7657,TCGA-58-8386,TCGA-22-5478,TCGA-22-5472,TCGA-43-5670,TCGA-60-2709,TCGA-22-5489,TCGA-77-8007,TCGA-22-5471,TCGA-22-4609,...,TCGA-56-7580,TCGA-43-6647,TCGA-90-6837,TCGA-56-8083,TCGA-51-4079,TCGA-56-7222,TCGA-22-4593,TCGA-51-4081,TCGA-56-8309,Normality
0,-274.67,-369.86,-670.22,-251.51,-413.72,-10.12,-52.97,52.62,-343.60,-303.01,...,-361.13,-69.59,-83.71,-1203.84,-90.36,-298.74,-190.81,-207.85,53.66,
1,79.96,70.27,17.01,-43.71,-7.08,-83.55,25.62,46.19,-83.50,-71.73,...,-15.98,-48.25,-271.68,-31.08,73.42,52.37,8.47,-92.09,202.71,
2,60.95,583.09,140.52,-15.65,1279.59,-12.80,1379.42,1109.77,16.54,-82.74,...,2196.90,105.37,1690.04,-2499.11,-86.11,146.60,256.90,263.15,531.88,
3,9.22,9.78,-4.92,60.96,5.51,17.36,7.99,8.76,26.66,30.51,...,63.39,51.38,56.23,33.58,31.40,21.28,28.58,0.99,43.75,
4,48.73,139.69,41.02,51.77,87.37,78.87,29.36,151.30,-216.69,68.07,...,90.58,32.45,56.09,-77.44,95.02,212.40,-3.71,-34.05,1.08,


In [10]:
# Looping over the samples
for i in range (len(diff)):
        # Execluding the normality column  
        sample = np.array(diff.iloc[i][:-1])
        
        # Checking the Normality of each sample by Shapiro Wilk test
        stat,pval = stats.shapiro(sample)
        
        # With a significance level of 0.05
        # if the p-value is more than 0.05, the data are normally distributed
        if pval > 0.05 :
            diff.at[i,'Normality'] = 'NORMAL'
            
        # Else, the sample is not normally distributed
        else: 
            diff.at[i,'Normality'] = 'NOT'
            
# Count the overall noramlly and not normally distributed data
norm = len(diff[diff['Normality'] == 'NORMAL'])
Not = len(diff[diff['Normality'] == 'NOT'])
print(f"Count of normally distributed {norm} and of not normally distributed {Not}")

Count of normally distributed 5626 and of not normally distributed 11649


## Independent 
- We would use the same Normality test but since the samples are independent, we can't check normality on the difference, instead we will check both cases independently

In [11]:
# Stack the data in a list for looping simplicity
dfs = [normal_x.dropna(), cancer_x.dropna()]
names = ['normal', 'cancer']

# Looping over every dataframe, Normal and cancer
for j,df in enumerate(dfs):
    # Create a column to carry the normality result of each sample
    df['normality'] = ''
    
    # Loop over the samples
    for i in range (len(df)):
        # Execlude the name, ID, and normality columns
        sample = np.array(df.iloc[i][2:-1])
        
        # Apply the Shapiro test 
        stat,pval = stats.shapiro(sample)
        
        # Same conditions as before applyes
        if pval > 0.05 :
            df.at[i,'normality'] = 'NORMAL'
        else: 
            df.at[i,'normality'] = 'NOT'
    # Count the overall noramlly and not normally distributed data
    norm = len(df[df['normality'] == 'NORMAL'])
    not_norm = len(df[df['normality'] == 'NOT'])
    print('dataframe {} has {} normally distrbuted genes and {} not normally distributed'.format(names[j], norm, not_norm))

dataframe normal has 6994 normally distrbuted genes and 10281 not normally distributed
dataframe cancer has 2028 normally distrbuted genes and 15247 not normally distributed


# 4. Hypothesis Testing

## 1. Paired Data
- Since nearly about 12K of the 17K samples are not normally distributed and only 5K are normally distributed, We will consider using non-normal hypothesis testing test for paired data, Wilcoxon test.

In [12]:
# Stack the data in a list for looping simplicity
data = [normal_x.dropna().copy(), cancer_x.dropna().copy()]

# Execlude unwanted raws
for df in data:
    df.drop('Hugo_Symbol',axis=1, inplace=True)
    df.drop('Entrez_Gene_Id',axis=1, inplace=True)
    print(df.shape)

(17275, 50)
(17275, 50)


In [13]:
# Apply the Wilcoxon test on normal and cancer samples
stat, p = stats.wilcoxon(data[0], data[1], axis=1)

# Create a DataFrame to hold the results
wilcoxon = pd.DataFrame(list(zip(normal_x.dropna().Hugo_Symbol, normal_x.dropna().Entrez_Gene_Id, diff.dropna().Normality, p, stat)))

# Initialize the columns headers
wilcoxon.columns =['Hugo_Symbol', 'Entrez_Gene_Id', 'normality', 'p', 'stat']

# A column to hold the hypothesis test result, True(Expressed) if p < 0.05, False if else.
wilcoxon['expressed'] = np.where(wilcoxon['p'] < 0.05, 'True', 'False')
print(len(wilcoxon))
wilcoxon.head(10)

17275


,Hugo_Symbol,Entrez_Gene_Id,normality,p,stat,expressed
0,HIST3H2A,92815,NOT,1.008929e-08,44.0,True
1,LIN7B,64130,NORMAL,2.384960e-01,494.0,False
2,LXN,56925,NOT,1.725267e-06,142.0,True
3,CNKSR2,22866,NORMAL,1.265880e-08,48.0,True
4,SCML1,6322,NORMAL,1.601853e-02,388.0,True
5,AC024592.12,0,NOT,6.735523e-02,448.0,False
6,GSDMD,79792,NOT,1.493082e-06,139.0,True
7,AKR1C1,1645,NOT,1.196552e-04,239.0,True
8,C3orf62,375341,NORMAL,4.583914e-08,55.0,True
9,CRISPLD2,83716,NOT,2.089327e-06,146.0,True


In [14]:
# For more Accurate(confident) decision, we'll apply FDR Correction
wilcoxon['fdr_hypothesis'] = fdr(p)[0].astype(str)
wilcoxon['fdr'] = fdr(p)[1]

# A column to notate if the fdr corrected any decision, True if corrected, False if else
wilcoxon['corrected'] = np.where(wilcoxon['expressed'] == wilcoxon['fdr_hypothesis'], 'False', 'True')

print('Count of genes corrected by the FDR Correction: ', len(wilcoxon[wilcoxon['corrected'] == 'True']))

Count of genes corrected by the FDR Correction:  226


In [15]:
wilcoxon.head(10)

,Hugo_Symbol,Entrez_Gene_Id,normality,p,stat,expressed,fdr_hypothesis,fdr,corrected
0,HIST3H2A,92815,NOT,1.008929e-08,44.0,True,True,4.682766e-08,False
1,LIN7B,64130,NORMAL,2.384960e-01,494.0,False,False,2.756234e-01,False
2,LXN,56925,NOT,1.725267e-06,142.0,True,True,4.290195e-06,False
3,CNKSR2,22866,NORMAL,1.265880e-08,48.0,True,True,5.630297e-08,False
4,SCML1,6322,NORMAL,1.601853e-02,388.0,True,True,2.223009e-02,False
5,AC024592.12,0,NOT,6.735523e-02,448.0,False,False,8.533013e-02,False
6,GSDMD,79792,NOT,1.493082e-06,139.0,True,True,3.754438e-06,False
7,AKR1C1,1645,NOT,1.196552e-04,239.0,True,True,2.221194e-04,False
8,C3orf62,375341,NORMAL,4.583914e-08,55.0,True,True,1.642545e-07,False
9,CRISPLD2,83716,NOT,2.089327e-06,146.0,True,True,5.108723e-06,False


#### Names of Genes Corrected by FDR Correction

In [16]:
wilcoxon[wilcoxon['corrected'] == 'True'].style.hide_index()
# wilcoxon[wilcoxon['corrected'] == 'True']['Hugo_Symbol']

Hugo_Symbol,Entrez_Gene_Id,normality,p,stat,expressed,fdr_hypothesis,fdr,corrected
PMEL,6490,NOT,0.039304,424.000000,True,False,0.051545,True
HS3ST4,9951,NOT,0.042292,321.000000,True,False,0.055140,True
GPR35,2859,NORMAL,0.040236,425.000000,True,False,0.052674,True
PLAC8L1,153770,NOT,0.042796,390.500000,True,False,0.055788,True
PRRC2B,84726,NORMAL,0.049479,434.000000,True,False,0.063949,True
MTPN,136319,NOT,0.048371,433.000000,True,False,0.062569,True
FAM163B,642968,NORMAL,0.043491,322.000000,True,False,0.056617,True
NPL,80896,NORMAL,0.041184,426.000000,True,False,0.053837,True
NPRL3,8131,NOT,0.042152,427.000000,True,False,0.054969,True
ELP5,23587,NORMAL,0.049478,434.000000,True,False,0.063949,True


### 2. Independent Data
- Also in this part, most of the samples(10K and 15K out of 17K) showed non-normal distribution, so we will consider using non-normal hypothesis testing test for independent data, Mannwhitneyu test.

In [17]:
# Apply the Wilcoxon Mannwhitneyu on normal and cancer samples
stat, p =  stats.mannwhitneyu(data[0], data[1], axis=1)

# Create a DataFrame to hold the results
mannwhitneyu = pd.DataFrame(list(zip(normal_x.dropna().Hugo_Symbol, normal_x.dropna().Entrez_Gene_Id, dfs[0].dropna().normality,dfs[1].dropna().normality, p, stat)))
# Initialize the columns headers
mannwhitneyu.columns =['Hugo_Symbol', 'Entrez_Gene_Id', 'Normality_NormalSample', 'Normality_CancerSample','p', 'stat']

# A column to hold the hypothesis test result, True(Expressed) if p < 0.05, False if else.
mannwhitneyu['expressed'] = np.where(mannwhitneyu['p'] < 0.05, 'True', 'False')

print(len(wilcoxon))
wilcoxon.head(10)


17275


,Hugo_Symbol,Entrez_Gene_Id,normality,p,stat,expressed,fdr_hypothesis,fdr,corrected
0,HIST3H2A,92815,NOT,1.008929e-08,44.0,True,True,4.682766e-08,False
1,LIN7B,64130,NORMAL,2.384960e-01,494.0,False,False,2.756234e-01,False
2,LXN,56925,NOT,1.725267e-06,142.0,True,True,4.290195e-06,False
3,CNKSR2,22866,NORMAL,1.265880e-08,48.0,True,True,5.630297e-08,False
4,SCML1,6322,NORMAL,1.601853e-02,388.0,True,True,2.223009e-02,False
5,AC024592.12,0,NOT,6.735523e-02,448.0,False,False,8.533013e-02,False
6,GSDMD,79792,NOT,1.493082e-06,139.0,True,True,3.754438e-06,False
7,AKR1C1,1645,NOT,1.196552e-04,239.0,True,True,2.221194e-04,False
8,C3orf62,375341,NORMAL,4.583914e-08,55.0,True,True,1.642545e-07,False
9,CRISPLD2,83716,NOT,2.089327e-06,146.0,True,True,5.108723e-06,False


In [18]:
# For more Accurate(confident) decision, we'll apply FDR Correction
mannwhitneyu['fdr_hypothesis'] = fdr(p)[0].astype(str)
mannwhitneyu['fdr'] = fdr(p)[1]

# A column to notate if the fdr corrected any decision, True if corrected, False if else
mannwhitneyu['corrected'] = np.where(mannwhitneyu['expressed'] == mannwhitneyu['fdr_hypothesis'], 'False', 'True')
mannwhitneyu['corrected'] = np.where(mannwhitneyu['expressed'] == mannwhitneyu['fdr_hypothesis'], 'False', 'True')
mannwhitneyu['corrected'] = np.where(mannwhitneyu['expressed'] == mannwhitneyu['fdr_hypothesis'], 'False', 'True')
mannwhitneyu['corrected'] = np.where(mannwhitneyu['expressed'] == mannwhitneyu['fdr_hypothesis'], 'False', 'True')


print('Count of genes corrected by the FDR Correction: ', len(mannwhitneyu[mannwhitneyu['corrected'] == 'True']))

Count of genes corrected by the FDR Correction:  189


#### Names of Genes Corrected by FDR Correction

In [19]:
mannwhitneyu[mannwhitneyu['corrected'] == 'True'].style.hide_index()
# wilcoxon[wilcoxon['corrected'] == 'True']['Hugo_Symbol']

Hugo_Symbol,Entrez_Gene_Id,Normality_NormalSample,Normality_CancerSample,p,stat,expressed,fdr_hypothesis,fdr,corrected
MAPK1,5594,NORMAL,NOT,0.044816,958.500000,True,False,0.058075,True
AL136218.1,0,NORMAL,NORMAL,0.043923,1542.500000,True,False,0.056981,True
ABT1,29777,NOT,NORMAL,0.045895,960.000000,True,False,0.059389,True
ARNTL,406,NOT,NOT,0.039265,1549.500000,True,False,0.051188,True
ITFG2,55846,NOT,NOT,0.044457,1542.000000,True,False,0.057630,True
AC115618.1,0,NORMAL,NOT,0.040602,1547.500000,True,False,0.052884,True
AC023590.1,0,NOT,NOT,0.045378,961.500000,True,False,0.058741,True
ZNF568,374900,NOT,NOT,0.043023,1544.000000,True,False,0.055881,True
RNF8,9025,NOT,NOT,0.048224,963.000000,True,False,0.062247,True
RNF4,6047,NORMAL,NOT,0.038583,949.500000,True,False,0.050361,True


In [23]:
# mannwhitneyu.to_csv("filtered/indep_fdr_2sided.csv")
# wilcoxon.to_csv("filtered/paired_fdr_2sided.csv")
# wilcoxon.to_csv("filtered/GSEA.txt",  sep="\t")

In [23]:
intersect = wilcoxon[mannwhitneyu['fdr_hypothesis'] == wilcoxon['fdr_hypothesis']]['Hugo_Symbol']
len(intersect)

16234

In [24]:
gsea =pd.read_csv('filtered/GSEA.txt', sep="\t", header=0)
gsea

,Unnamed: 0,Hugo_Symbol,Entrez_Gene_Id,normality,p,stat,expressed,fdr_hypothesis,fdr,corrected
0,0,HIST3H2A,92815,NOT,1.008929e-08,44.0,True,True,4.682766e-08,False
1,1,LIN7B,64130,NORMAL,2.384960e-01,494.0,False,False,2.756234e-01,False
2,2,LXN,56925,NOT,1.725267e-06,142.0,True,True,4.290195e-06,False
3,3,CNKSR2,22866,NORMAL,1.265880e-08,48.0,True,True,5.630297e-08,False
4,4,SCML1,6322,NORMAL,1.601853e-02,388.0,True,True,2.223009e-02,False
...,...,...,...,...,...,...,...,...,...,...
17270,17270,ZNF521,25925,NaN,8.775508e-06,177.0,True,True,1.934868e-05,False
17271,17271,SPINT2,10653,NaN,4.778330e-09,31.0,True,True,2.616344e-08,False
17272,17272,HAVCR2,84868,NaN,3.366814e-09,25.0,True,True,2.030077e-08,False
17273,17273,CTD-2116N17.1,0,NaN,2.964334e-09,16.0,True,True,1.858329e-08,False


In [31]:
gsea2 = normal_x.dropna().copy()
gsea2=gsea2.rename(columns={"Hugo_Symbol": "Name ", "Entrez_Gene_Id": "Description"})
# gsea2 = gsea2.dropna().iloc[: , 1:]
gsea2

,Name,Description,TCGA-43-7657,TCGA-58-8386,TCGA-22-5478,TCGA-22-5472,TCGA-43-5670,TCGA-60-2709,TCGA-22-5489,TCGA-77-8007,...,TCGA-85-7710,TCGA-56-7580,TCGA-43-6647,TCGA-90-6837,TCGA-56-8083,TCGA-51-4079,TCGA-56-7222,TCGA-22-4593,TCGA-51-4081,TCGA-56-8309
0,HIST3H2A,92815,62.12,130.60,33.06,35.50,73.03,60.39,92.05,66.65,...,90.77,59.55,40.07,22.92,29.91,82.29,4.70,37.32,43.63,77.25
1,LIN7B,64130,185.11,283.05,119.26,169.07,165.57,161.02,131.51,198.47,...,185.11,119.26,102.97,123.50,264.03,194.36,166.73,105.15,185.11,356.05
2,LXN,56925,909.17,819.30,412.00,743.43,1340.84,607.87,1709.26,1709.26,...,813.63,2400.97,543.96,2193.99,540.19,521.76,253.23,764.36,518.15,878.17
3,CNKSR2,22866,41.81,18.29,40.93,67.12,54.72,29.27,20.26,23.76,...,34.51,70.01,57.49,57.89,67.12,34.51,22.10,31.90,28.24,49.91
4,SCML1,6322,133.36,214.27,108.14,109.66,190.34,211.31,96.01,208.38,...,251.48,209.84,120.10,109.66,155.50,162.14,277.20,86.43,164.42,155.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19641,ZNF521,25925,215.77,148.09,83.45,232.94,167.90,80.01,161.02,104.42,...,145.02,202.66,111.99,371.22,154.42,289.02,32.59,191.67,204.07,88.26
19642,SPINT2,10653,6792.79,5441.30,5831.91,5329.30,5711.87,6164.49,8134.41,6516.03,...,3955.48,4869.99,6935.54,4836.35,7230.10,6250.56,6840.04,5292.48,3928.15,7130.55
19643,HAVCR2,84868,423.61,529.06,660.68,620.67,518.15,848.22,366.09,1073.91,...,559.28,579.04,420.68,366.09,334.46,258.57,329.84,1030.12,665.29,633.73
19646,CTD-2116N17.1,0,0.00,0.00,0.99,3.59,2.23,3.35,0.00,7.34,...,6.94,1.00,0.00,4.58,0.96,3.06,1.83,1.77,3.63,2.56


In [32]:
gsea2.to_csv("filtered/GSEA.txt",  sep="\t", index=False)